# Olist E-Commerce Website Data Analysis 

### Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.preprocessing import scale
from sklearn.preprocessing import normalize
from sklearn.metrics import r2_score, confusion_matrix
import datetime
import random

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%matplotlib inline
sns.set()
pd.options.display.max_columns = 100

# Fonctions d'automatisation

In [51]:
# take df, make copy, show info() and return the dataframe
def copy_get_info_df(df):
    new_df = df.copy()
    new_df.info()
    print('total missing values : ', new_df.insull().sum())
    print('the shape is : ', new_df.shape)
    return new_df
#-------------------------------------------------------------------
# pass list of dataframes and get all the infos, shape, missing values at once 
def get_all_df_info(dataframes):
    df_names = ['customer data :','order items : ','order payments : ','order reviews :','orders : ','products : ','sellers : ','product translation :']
    for name,df in zip(df_names,dataframes):
        print('\n INFORMATION ABOUT THE DATASET ', name,'\n')
        df.info()
        print('Total missing values : ', df.isnull().sum())
        print('Shape : ', df.shape)
        print('\n')
#---------------------------------------------------------------------
        
        

In [52]:
test = [customers_dataset_df, order_items_df]
get_all_df_info(test)


 INFORMATION ABOUT THE DATASET  customer data : 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
customer_id                 99441 non-null object
customer_unique_id          99441 non-null object
customer_zip_code_prefix    99441 non-null int64
customer_city               99441 non-null object
customer_state              99441 non-null object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB
Total missing values :  customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64
Shape :  (99441, 5)



 INFORMATION ABOUT THE DATASET  order items :  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
order_id               112650 non-null object
order_item_id          112650 non-null int64
product_id             112650 non-null object
seller_id              112650 non-nu

### Loading the Data

#### customers_dataset

In [2]:
# no nulls values
customers_dataset_df = pd.read_csv('./brazilian-ecommerce/olist_customers_dataset.csv')
customers_dataset_df_clean = customers_dataset_df.copy()

customers_dataset_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
customer_id                 99441 non-null object
customer_unique_id          99441 non-null object
customer_zip_code_prefix    99441 non-null int64
customer_city               99441 non-null object
customer_state              99441 non-null object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


In [20]:
customers_dataset_dfname = [ k for k,v in locals().iteritems() if v == customers_dataset_df][0]
 


AttributeError: 'dict' object has no attribute 'iteritems'

#### order_items

In [3]:
# no nulls

order_items_df = pd.read_csv('./brazilian-ecommerce/olist_order_items_dataset.csv')
order_items_df_clean = order_items_df.copy()

order_items_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
order_id               112650 non-null object
order_item_id          112650 non-null int64
product_id             112650 non-null object
seller_id              112650 non-null object
shipping_limit_date    112650 non-null object
price                  112650 non-null float64
freight_value          112650 non-null float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB


#### order_payments

In [4]:
# no nulls
order_payments_dataset_df = pd.read_csv('./brazilian-ecommerce/olist_order_payments_dataset.csv')
order_payments_dataset_df_clean = order_payments_dataset_df.copy()

order_payments_dataset_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
order_id                103886 non-null object
payment_sequential      103886 non-null int64
payment_type            103886 non-null object
payment_installments    103886 non-null int64
payment_value           103886 non-null float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB


#### order_reviews_dataset

In [5]:
# review_comment_title and review_comment_message fields are empty which is fine, we will not be using them in out analysis so
# we will drop the columns in addition to review_creation_date and review_answer_timestamp columns
order_reviews_dataset_df = pd.read_csv('./brazilian-ecommerce/olist_order_reviews_dataset.csv')
order_reviews_dataset_df_clean = order_reviews_dataset_df.copy()

order_reviews_dataset_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 7 columns):
review_id                  100000 non-null object
order_id                   100000 non-null object
review_score               100000 non-null int64
review_comment_title       11715 non-null object
review_comment_message     41753 non-null object
review_creation_date       100000 non-null object
review_answer_timestamp    100000 non-null object
dtypes: int64(1), object(6)
memory usage: 5.3+ MB


In [6]:
stop there

SyntaxError: invalid syntax (<ipython-input-6-10b128d7a1de>, line 1)

In [ ]:
# we drop also review creation date and answer date we dont need those
order_reviews_dataset_df_clean.drop(columns = ['review_comment_title','review_comment_message','review_creation_date','review_answer_timestamp'], inplace = True)

In [ ]:
order_reviews_dataset_df_clean.head()

In [ ]:
order_reviews_dataset_df_clean.info()

In [ ]:
order_reviews_dataset_df_clean.describe()

#### orders_dataset

In [ ]:
orders_dataset_df = pd.read_csv('./brazilian-ecommerce/olist_orders_dataset.csv')
orders_dataset_df_clean = orders_dataset_df.copy()
orders_dataset_df.info()


In [ ]:
orders_dataset_df_clean.head()

In [ ]:
# 1.we will be assessing if our estimation on the order_estimated_delivery_date is correct most of the time or not, so in this case
# we will create a df that has only the timestamps and dates and we will be dropping this nan values to do so, that is so we 
# do not loose data for other operations from the original dataframe.
# 2. we need to change  the data type of the dates to datetime.
order_datetime_list = ['order_purchase_timestamp', 'order_approved_at',\
                       'order_delivered_carrier_date', 'order_delivered_customer_date',\
                       'order_estimated_delivery_date']


orders_dataset_df_clean[order_datetime_list] = orders_dataset_df_clean[order_datetime_list].apply(pd.to_datetime, errors = 'coerce')
orders_dataset_df_clean.head()

In [ ]:
# so now we have the datetime type
orders_dataset_df_clean.info()

In [ ]:
# checking valid values for order_status
orders_dataset_df_clean.order_status.unique()

In [ ]:
# checking if for the delivered items we have all the timings, the invoiced only can have nan values it is fine for the dates variables.
# we can see that we still have missing dates for already delivered orders, we can investigate closely in the company to see why we are missing these dates.

orders_dataset_df_clean.query("order_status in 'delivered'").isnull().sum()

In [ ]:
# getting the indexes of the columns that have a delivered order_status and a null order_approved_at because it does not make sense
delivered_and_not_approved_indexes = orders_dataset_df_clean[orders_dataset_df_clean['order_approved_at'].isnull()].query("order_status in 'delivered'").index

In [ ]:
orders_dataset_df_clean.drop(inplace = True, index = delivered_and_not_approved_indexes)

In [ ]:
orders_dataset_df_clean.query("order_status in 'delivered'").isnull().sum()

#### products_dataset

In [ ]:
products_dataset_df = pd.read_csv('./brazilian-ecommerce/olist_products_dataset.csv')
products_dataset_df_clean = products_dataset_df.copy()

products_dataset_df.info()

In [ ]:
# we will be dropping the rows that have missing product_category_name, product_name_lenght, product_description_lenght, product_photos_qty
# which are 610 rows of 32951
products_dataset_df_clean[products_dataset_df_clean['product_category_name'].isnull()].shape
null_products_indexes = products_dataset_df_clean[products_dataset_df_clean['product_category_name'].isnull()].index
products_dataset_df_clean.drop(inplace = True, index = null_products_indexes)

In [ ]:
# testing
print(products_dataset_df_clean.shape[0],products_dataset_df.shape[0])

In [ ]:
# 1 row missing dimensions, we will drop it also since it has negligible effect on our analysis
products_dataset_df_clean.info()

In [ ]:
additional_products_missing_g_indexes = products_dataset_df_clean[products_dataset_df_clean['product_weight_g'].isnull()].index

In [ ]:
products_dataset_df_clean.drop(inplace = True, index = additional_products_missing_g_indexes)

In [ ]:
products_dataset_df_clean.info()

In [ ]:
products_dataset_df_clean.describe()

#### sellers_dataset

In [ ]:
# We don't have missing values !
sellers_dataset_df = pd.read_csv('./brazilian-ecommerce/olist_sellers_dataset.csv')
sellers_dataset_df_clean = sellers_dataset_df.copy()

sellers_dataset_df.info()

#### product_category_name_translation

In [ ]:
product_category_name_translation_df = pd.read_csv('./brazilian-ecommerce/product_category_name_translation.csv')
product_category_name_translation_df.info()

In [ ]:
# creating a master dataframe, that has all the clean dataframes to get more insights when we want to visualize, model and predict
master_df = orders_dataset_df_clean.copy()

master_df = master_df.merge(customers_dataset_df_clean, on='customer_id', indicator=True)

In [ ]:
master_df = master_df.merge(order_reviews_dataset_df_clean, on='order_id')

In [ ]:
master_df = master_df.merge(order_payments_dataset_df_clean, on='order_id')

In [ ]:
master_df = master_df.merge(order_items_df_clean, on = 'order_id')

In [ ]:
master_df = master_df.merge(products_dataset_df_clean, on = 'product_id')

In [ ]:
master_df = master_df.merge(sellers_dataset_df_clean, on = 'seller_id')

In [ ]:
master_df.shape

# Exploratory Data Visualization

#### In this section we will be answering inquiries using visualizations of data insights using univariable, bivariable and multivariable plots

In [ ]:
master_df.shape

In [ ]:
# we only have two columns having null values, order_delivered_carrier_date and order_delivered_customer_date  
master_df.isnull().sum()

In [ ]:
plt.figure(figsize=(12,10))
sns.heatmap(master_df.corr().round(2), xticklabels=master_df.corr().columns, yticklabels=master_df.corr().columns, cmap='Spectral', center=0, annot=True)

plt.title('Corrélations entres les features', fontsize=22)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

In [ ]:
# I prefer round all correlations > 0.5 and make them = 1 so we can identify more clearely what is going on 
corr_df = master_df.corr().round(2)
plt.figure(figsize=(12,10))
sns.heatmap(abs(corr_df)>0.5 , xticklabels=master_df.corr().columns, yticklabels=master_df.corr().columns, cmap='Spectral', center=0, annot=True)

plt.title('corrélations simplifié', fontsize=22)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

In [ ]:
plt.figure(figsize=(14,8))

sns.regplot(x=master_df['product_weight_g'], y=master_df['freight_value'], scatter_kws={'alpha':0.7, 'color':'coral'},\
           line_kws={'color':'red'});

plt.xlim(0, 30000)

plt.title('Correlation entre le poids du produit et la valeur Fret', fontsize=22)
plt.xlabel('Product Weight'.title(), fontsize=15)
plt.ylabel('Freight Value'.title(), fontsize=15)

plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()



In [ ]:
# Distributions for some features
master_df.hist(figsize=(15,20), color='skyblue');

## Univariable Plots

* Type de payements 
* Range des prix de payement 
* Villes qui achetent le plus 
* fréquence d'achat par categorie de produit

In [ ]:
payment_type_count_plot_df = master_df[['payment_type','customer_id']].groupby(['customer_id', 'payment_type']).count().reset_index()

In [ ]:
# Clients prefer the use of Credit card clearely

plt.figure(figsize=(14,8))

sns.countplot(payment_type_count_plot_df['payment_type']);

plt.title('What is the most frequent payment type?'.title() , fontsize=20);
plt.ylabel('count'.title(), fontsize=14);
plt.xlabel('payment type'.title(), fontsize=14);

In [ ]:
# Payement value frequency

plt.figure(figsize=(14,8))

# we use .sum() because we have the same clients sometimes so we avoid noise and confusion
min_order_payment_value = min(master_df.groupby('customer_id').sum().reset_index()['payment_value'])
max_order_payment_value = max(master_df.groupby('customer_id').sum().reset_index()['payment_value'])

bins= np.linspace(start=min_order_payment_value, stop=1400, num=50)
plt.hist(x=(master_df.groupby('customer_id').sum().reset_index()['payment_value']), bins=bins,color='skyblue');

plt.title('payment value distribution per customer'.title() , fontsize=20);
plt.ylabel('count'.title(), fontsize=14, rotation=90);
plt.xlabel('payment value'.title(), fontsize=14);


In [ ]:
customer_state_count_df = master_df.groupby(['customer_id','customer_state']).count().reset_index()

In [ ]:
plt.figure(figsize=(14,8))
sns.countplot(customer_state_count_df.customer_state, order=customers_dataset_df.customer_state.value_counts().index);

plt.title('Most frequent customer state'.title(), fontsize=20);
plt.ylabel('count'.title(), fontsize=14);
plt.xlabel('customer state'.title(), fontsize=14);

plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

In [ ]:
# adding the translated product name category to the data_frame to use it in the following plots.
master_df_with_product_categories = master_df.merge(product_category_name_translation_df, how='left', on='product_category_name');

In [ ]:
plt.figure(figsize=(14,8))

sns.countplot(master_df_with_product_categories.product_category_name_english,#
              order=master_df_with_product_categories.product_category_name_english.value_counts().index)

plt.title('Most in demand product category'.title(), fontsize=20);
plt.ylabel('count'.title(), fontsize=14);
plt.xlabel('product category'.title(), fontsize=14);

plt.xticks(rotation=90, fontsize=10);
plt.yticks(fontsize=12)
plt.show()

## Bivariable plots

* Mean score vs products category (Bar plot)
* Mean score vs city (Bar plot)
* Order Price vs Freight Price (Scatter plot)

In [ ]:
mean_score_vs_cat_df = master_df_with_product_categories.groupby(['product_category_name_english']).mean()['review_score'].reset_index()
mean_score_vs_cat_df

In [ ]:
# The on average the best product category rating wise is that of cds_dvds_musicals, and the securit_and_services seems to have
# the lowest rating.

plt.figure(figsize=(14,8))

sns.barplot(data = master_df_with_product_categories, x='product_category_name_english', y='review_score',
            order = mean_score_vs_cat_df.sort_values('review_score', ascending=False)['product_category_name_english'].values
            ,errcolor = 'grey')

plt.title('Most highly rated product category'.title(), fontsize=20);
plt.ylabel('Average Review Score'.title(), fontsize=14);
plt.xlabel('Product Category'.title(), fontsize=14);

plt.xticks(rotation=90, fontsize= 10);
plt.yticks(fontsize=12)
plt.show()

In [ ]:
# The nicest state ?

plt.figure(figsize=(14,8))

sns.barplot(data = master_df_with_product_categories, x='customer_state', y='review_score',\
           order = master_df_with_product_categories.groupby('customer_state').mean()\
            .reset_index().sort_values('review_score', ascending=False)['customer_state'].values,
           errcolor = 'grey');


plt.title('Best reviews by states'.title(), fontsize=20);
plt.ylabel('Average Review Score'.title(), fontsize=14);
plt.xlabel('customer state'.title(), fontsize=14);

plt.xticks(rotation=0, fontsize= 12);
plt.yticks(fontsize=12)
plt.show()

In [ ]:
# relationship between the price of the order and the price of the freight?

payment_vs_freight_df = master_df_with_product_categories.groupby('customer_id').sum().reset_index()[['payment_value','freight_value']]
payment_vs_freight_df


In [ ]:
plt.figure(figsize=(14,8))
sns.regplot(data=payment_vs_freight_df, x=payment_vs_freight_df.payment_value, y=payment_vs_freight_df.freight_value,
            scatter_kws={'alpha':0.7, 'color':'skyblue'}, line_kws={'color':'red'});
plt.xlim(0,20000);

plt.title('Correlation between Payment and Freight value'.title(), fontsize=20);
plt.ylabel('Freight value'.title(), fontsize=14);
plt.xlabel('Payment Value'.title(), fontsize=14);

plt.xticks(rotation=0, fontsize= 12);
plt.yticks(fontsize=12)
plt.show()

## Multivariable Plots
* Payment value vs. month vs. year
* we want to see the Evolution over time to understand better

In [ ]:
master_df_with_product_categories['year'] = pd.DatetimeIndex(master_df_with_product_categories['order_purchase_timestamp']).year

In [ ]:
master_df_with_product_categories['month'] = pd.DatetimeIndex(master_df_with_product_categories['order_purchase_timestamp']).month 

In [ ]:
master_df_with_product_categories.groupby(['year','month']).sum()['payment_value'] #extremely powerful groupby

In [ ]:
multivariable_plot_df = master_df_with_product_categories.groupby(['year','month']).sum()['payment_value'].reset_index()
multivariable_plot_df.head()


In [ ]:
g = sns.FacetGrid(data=multivariable_plot_df, row = 'year', height=7, aspect=2.5, sharey=True)

g = g.map(plt.bar, 'month', 'payment_value', color='skyblue')
g.set_titles( size=20);
g.set_ylabels('Payment Value'.title(), size=14);
g.set_xlabels('months'.title(), size=14);



# Feature Engineering ( creation de nouvelles features pour aider l'algorithme de clustering ) 

In [ ]:
master_df_with_product_categories['order_delivered_customer_date'].fillna(method='bfill',inplace=True)

In [ ]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()


master_df1  = master_df_with_product_categories.copy()
# 3 features : order_status, customer_state, payment_type
master_df1['order_status'] = le.fit_transform(master_df1['order_status'])
master_df1['customer_state'] = le.fit_transform(master_df1['customer_state'])
master_df1['payment_type'] = le.fit_transform(master_df1['payment_type'])

In [ ]:
# feature  : time it took to deliver 
master_df1['time_elapsed_delivery'] = master_df1['order_delivered_customer_date'] - master_df1['order_purchase_timestamp']
master_df1['time_elapsed_delivery'] = master_df1.time_elapsed_delivery.dt.days

## Recency

In [ ]:
#creates a generic user dataframe to keep CustomerID and new segmentation scores
df_user = pd.DataFrame(master_df1['customer_unique_id'])
df_user.columns = ['customer_unique_id']

#gets the max purchase date for each customer and create a dataframe with it
df_max_purchase = master_df1.groupby('customer_unique_id').order_purchase_timestamp.max().reset_index()
df_max_purchase.columns = ['customer_unique_id', 'MaxPurchaseDate']

#we take our observation point as the max purchase date in our dataset
df_max_purchase['Recency'] = (df_max_purchase['MaxPurchaseDate'].max() - df_max_purchase['MaxPurchaseDate']).dt.days

#merge this dataframe to our new user dataframe
df_user = pd.merge(df_user, df_max_purchase[['customer_unique_id','Recency']], on='customer_unique_id')

df_user= df_user.groupby('customer_unique_id').mean().reset_index()

#merge it to master df
master_df1 = pd.merge(master_df1, df_user, on='customer_unique_id')
df_user.head()

## Frequency 

In [ ]:
#get order counts for each user and create a dataframe with it
df_frequency = master_df1.groupby('customer_unique_id').order_purchase_timestamp.count().reset_index()
df_frequency.columns = ['customer_unique_id','Frequency']

#add this data to our main dataframe
master_df1 = pd.merge(master_df1, df_frequency, on='customer_unique_id')
master_df1

In [ ]:
master_df1.info()

# Machine Learning Applications



## (Unsupervised Learning)

In [ ]:
        # selecting numeric columns
"""creating dataframe that has only numeric columns then extract them easy"""
numeric_df = master_df1.select_dtypes(exclude=[object,'datetime'])
numeric_columns = list(numeric_df.columns.values)
numeric_columns

In [ ]:
columns_to_remove = ['_merge', 'product_height_cm', 'product_width_cm', 'product_length_cm', 'payment_installments', 'order_item_id', 'price', 'month','order_item_id']

numeric_list_clean = [element for element in numeric_columns if element not in columns_to_remove]


In [ ]:
numeric_list_clean.append('customer_unique_id')

In [ ]:
master_df1[numeric_list_clean]

In [ ]:
# Very high level of groupby use ! 
# we are grouping by customer ids and aggregating every column on what it should be 
clustering_customers_df = master_df1[numeric_list_clean]
clustering_customers_df = clustering_customers_df.groupby('customer_unique_id').agg({ 'review_score': 'mean', 'payment_value':'sum','seller_zip_code_prefix':'max',
          'product_weight_g': 'mean','product_photos_qty':'mean',
           'product_name_lenght':'mean','Recency':'max','Frequency':'sum'}).reset_index()

clustering_customers_df.info()

In [ ]:
plt.figure(figsize=(12,10))
sns.heatmap(clustering_customers_df.corr().round(2), xticklabels=clustering_customers_df.corr().columns, yticklabels=clustering_customers_df.corr().columns, cmap='Spectral',vmin=-1, vmax=1, annot=True)

plt.title('Corrélations entres les features', fontsize=22)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

## Scale numeric columns before K-means

In [ ]:
# selecting numeric columns
numeric_df1 = clustering_customers_df.select_dtypes(exclude=[object])
num_columns = list(numeric_df1.columns.values)
num_columns

In [ ]:
from sklearn.preprocessing import StandardScaler
#instantiate scaler
scaler = StandardScaler()
data_scaled = scaler.fit_transform(numeric_df1)
data_scaled.shape

In [ ]:
# slicing data for T-SNE
sliced_data = numeric_df1.sample(10000,random_state=9)
sclaed_sliced_data = scaler.transform(sliced_data)
sliced_data_df = pd.DataFrame(data = sclaed_sliced_data, columns=sliced_data.columns)
sliced_data_df.head()

# Unsupervised Learning using K-means

In [ ]:
# KMeans
kmeans = KMeans(4,random_state=9)
kmeans.fit(data_scaled)
predicted_df = sliced_data_df.copy()
predicted_df['cluster_pred'] = kmeans.predict(sliced_data_df) ## clusters remplace

In [ ]:
predicted_df

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2,init='pca',n_jobs=-1,random_state=9,verbose=1,perplexity=50)
data_embedded = tsne.fit_transform(sliced_data_df)

In [ ]:
plt.figure(figsize=(14,8))
ax = sns.scatterplot(x=data_embedded[:,0], y=data_embedded[:,1], alpha=0.8,\
                x_jitter=0.5, y_jitter=0.5, hue=predicted_df['cluster_pred'],\
                palette=sns.color_palette("hls", n_colors=4));


plt.setp(ax.get_legend().get_title(), fontsize='25')
plt.setp(ax.get_legend().get_texts(), fontsize='20')
plt.title('Clustering Customers'.title(), fontsize=20);

plt.xticks(rotation=0, fontsize= 12);
plt.yticks(fontsize=12)
plt.show()

## coef shilouaite

In [ ]:
from sklearn.metrics import silhouette_score
silh = []
for i in range(2,10):
    kmeans = KMeans(i,n_jobs=-1,n_init=1, verbose=3)
    kmeans.fit(sliced_data_df)
    silhouhaite = silhouette_score(sliced_data_df, kmeans.labels_)
    silh.append(silhouhaite)

In [ ]:
# 3 seems to have a good change in wcss, we will try it.
plt.figure(figsize=(14,8))

plt.plot(range(2,10), silh, linewidth=4, marker='o');


In [ ]:
zezeze

## Implementing the Elbow Method to choose the number of clusters